In [1]:
import pandas as pd
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm, metrics
from sklearn.metrics import accuracy_score

np.random.seed(500)
nlp = spacy.load("fr_core_news_md")

In [2]:
TRAIN_PATH = "datasets/articles/train_text_dataset.csv"
TEST_PATH = "datasets/articles/test_text_dataset.csv"

fields = ["label", "article"]

train_df = pd.read_csv(TRAIN_PATH, usecols=fields)
test_df = pd.read_csv(TEST_PATH, usecols=fields)

In [3]:
# Basic cleansing
def cleansing(doc):
    # Remove stop words
    doc = [token for token in doc if not token.is_stop]
    return doc

def keep_specific_pos(doc, pos=["ADV", "ADJ", "VERB", "NOUN"]):
    doc = [token for token in doc if token.pos_ in pos]
    return doc

def preprocess(data):
    docs = list(nlp.pipe(data))
    preprocess_docs = [keep_specific_pos(cleansing(doc)) for doc in docs]
    # Doc -> Text (+ lemmatization)
    output_texts = [" ".join([token.lemma_ for token in doc]) for doc in preprocess_docs]
    return output_texts

In [43]:
# x_train = preprocess([str(text) for text in train_df["article"].values])
# np.save("datasets/articles/x_train.npy", x_train)
x_train = list(np.load("datasets/articles/x_train.npy"))
STOP_LEMMA = ["pourcent", "climatique",
"pourcent",
"émission",
"climat",
"pays",
"énergie",
"changement",
"réchauffement",
"année",
"greenpeace"]
for i in range(len(x_train)):
    for stop_lemma in STOP_LEMMA:
        x_train[i] = x_train[i].replace(stop_lemma, '')

# x_test = preprocess([str(text) for text in test_df["article"].values])
# np.save("datasets/articles/x_test.npy", x_test)
x_test = list(np.load("datasets/articles/x_test.npy"))
for i in range(len(x_test)):
    for stop_lemma in STOP_LEMMA:
        x_test[i] = x_test[i].replace(stop_lemma, '')

In [44]:
y_train, y_test = train_df["label"].values - 1, test_df["label"].values - 1

In [45]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(x_train + x_test)
Train_X_Tfidf = Tfidf_vect.transform(x_train)
Test_X_Tfidf = Tfidf_vect.transform(x_test)

In [46]:
print(Tfidf_vect.vocabulary_)

{'renouvelable': 3829, 'combiner': 848, 'usage': 4646, 'intelligent': 2378, 'fournir': 1938, 'moitié': 2816, 'besoin': 518, 'énergétique': 4903, 'mondial': 2821, 'ici': 2177, 'présent': 3581, 'rapport': 3707, 'publier': 3613, 'révolution': 4076, 'avenir': 455, 'propre': 3516, 'durable': 1371, 'montre': 2835, 'économiquement': 4842, 'réduire': 3989, 'co2': 819, 'cours': 1124, 'prochain': 3455, 'conclure': 934, 'recours': 3754, 'massif': 2710, 'source': 4256, 'techniquement': 4443, 'manque': 2679, 'soutien': 4266, 'réglementaire': 4007, 'adéquat': 139, 'apparaître': 295, 'aujourd': 421, 'hui': 2150, 'évident': 4983, 'monde': 2820, 'impérativement': 2242, 'inscrire': 2350, 'rupture': 3949, 'net': 2930, 'orientation': 3101, 'mode': 2803, 'production': 3466, 'consommation': 998, 'passé': 3206, 'impératif': 2241, 'autant': 427, 'nécessité': 2992, 'parvenir': 3200, 'forme': 1916, 'sécurité': 4395, 'urgence': 4643, 'maîtriser': 2726, 'pollution': 3366, 'local': 2590, 'devoir': 1239, 'utilisati

In [47]:
id_to_word = {v: k for k, v in Tfidf_vect.vocabulary_.items()}

In [48]:
print(Train_X_Tfidf)

  (0, 4988)	0.07139403686052345
  (0, 4983)	0.10749214026067981
  (0, 4903)	0.4222003210215552
  (0, 4891)	0.13869935659316499
  (0, 4842)	0.13691333959409804
  (0, 4653)	0.16838233502403327
  (0, 4646)	0.09615796653491733
  (0, 4643)	0.07703536857673303
  (0, 4444)	0.0941598630013223
  (0, 4443)	0.14376731974181114
  (0, 4395)	0.09149627015559465
  (0, 4266)	0.09751177323938391
  (0, 4256)	0.07533587937419416
  (0, 4164)	0.056557371789606924
  (0, 4076)	0.11548588091561138
  (0, 4007)	0.13217507689064517
  (0, 3989)	0.11815947293007611
  (0, 3949)	0.1344395516229485
  (0, 3829)	0.16602245291506817
  (0, 3754)	0.10227564467510392
  (0, 3707)	0.05102360092358712
  (0, 3613)	0.06144860179388578
  (0, 3610)	0.09541402064843849
  (0, 3603)	0.06737290945717597
  (0, 3581)	0.16245041891693426
  :	:
  (1557, 1351)	0.07762012362203907
  (1557, 1345)	0.05133634880450817
  (1557, 1322)	0.0468626778919244
  (1557, 1309)	0.1408582918628759
  (1557, 1284)	0.06905802618489541
  (1557, 1273)	0.048585

In [49]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto', class_weight='balanced')
SVM.fit(Train_X_Tfidf,y_train)# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)
confusion_matrix = metrics.classification_report(y_test, predictions_SVM, zero_division=0)
report = metrics.confusion_matrix(y_test, predictions_SVM)
print(confusion_matrix)
print(report)

SVM Accuracy Score ->  79.70149253731343
              precision    recall  f1-score   support

           0       0.91      0.87      0.89       223
           1       0.40      0.54      0.46        37
           2       0.75      0.72      0.73        75

    accuracy                           0.80       335
   macro avg       0.69      0.71      0.69       335
weighted avg       0.82      0.80      0.80       335

[[193  18  12]
 [ 11  20   6]
 [  9  12  54]]


In [50]:
for i in range(SVM.support_.shape[0]):
    label = SVM.predict(SVM.support_vectors_.getrow(i))
    word = id_to_word[SVM.support_vectors_.getrow(i).argmax()]
    print(f'{label}: {word}')

[0]: énergétique
[0]: siège
[0]: protocole
[0]: ville
[0]: pollution
[0]: sanctuaire
[0]: quartier
[0]: république
[0]: donnée
[0]: extraire
[0]: réacteur
[0]: société
[0]: épisode
[1]: courage
[1]: avaaz
[0]: action
[0]: sécurité
[0]: santé
[1]: social
[1]: européen
[0]: indigène
[0]: tribune
[0]: victoire
[1]: bois
[0]: sénat
[0]: degré
[0]: minier
[0]: accord
[0]: partenariat
[0]: accueillir
[1]: malus
[0]: indonésien
[0]: g20
[0]: marée
[0]: alpe
[0]: glacier
[0]: iceberg
[0]: marin
[0]: convention
[0]: eau
[1]: accord
[0]: degré
[0]: véhicule
[0]: migrant
[0]: constitutionnel
[0]: gramme
[0]: glacier
[0]: électricité
[0]: glacier
[0]: bâtiment
[0]: droit
[0]: manifestant
[0]: co2
[0]: croissance
[0]: injecter
[0]: passager
[0]: chaleur
[0]: vendredi
[0]: vidéo
[0]: poumon
[0]: scénario
[0]: humidité
[0]: croissance
[0]: voile
[0]: éolienne
[0]: bouteille
[0]: gel
[0]: euro
[1]: compensation
[0]: rupture
[0]: urgent
[0]: ours
[0]: sommet
[1]: mouvement
[0]: lac
[0]: recyclage
[0]: 

In [51]:
arg_max = SVM.support_vectors_.sum(0).argsort().transpose()

for i in range(1, len(arg_max) + 1):
    id = int(arg_max[-i])
    print(id_to_word[id])

co2
degré
an
température
bien
gaz
rapport
carbone
européen
grand
faire
politique
mondial
monde
nucléaire
dernier
objectif
scientifique
pouvoir
eau
devoir
français
accord
terre
ici
forêt
serre
niveau
gouvernement
état
mettre
énergétique
falloir
prendre
planète
nouveau
projet
million
environnement
étude
action
non
entreprise
jour
glace
réduire
humain
président
milliard
fois
fin
aujourd
hui
mesure
vouloir
voir
écologique
développement
mois
agir
international
électricité
charbon
question
économique
public
venir
passer
atteindre
production
temps
région
renouvelable
permettre
transition
moyen
réduction
loi
fossile
euro
bon
ministre
important
augmentation
océan
modèle
crise
produire
nord
risque
prix
citoyen
cause
vie
activité
environnemental
point
national
population
petit
siècle
publier
personne
lutte
hausse
système
américain
total
face
exemple
période
naturel
problème
espèce
groupe
engagement
fort
donnée
consommation
article
pétrole
zone
centrale
etat
produit
cours
conséquence
chaleur
impac